- Read the Excel spreadsheet into a Pandas data frame. The index should contain state names (well, abbreviations) from the "State" column, without the footnotes. Remove the footnote lines at the end. You can remove the columns containing ">" symbols. You can treat both "none" and "n.a." as 0 values. Cells that contain text (i.e., not numbers) can be turned into 0.
- Which state has the highest rate for single filers? Which states have no income tax at all?

In [1]:
import pandas as pd

filename = '/Users/reuven/BambooWeekly/notebooks/data/bw-111.xlsx'

In [46]:
df = (
    pd
    .read_excel(filename, 
                header=[0, 1],
                nrows=217)
    
      .replace(to_replace='\(.*\)', value='', regex=True)
      .replace(to_replace='none', value='0')
      .replace(to_replace='n.a.', value='0')
      .replace(to_replace='^\s*$', value=pd.NA, regex=True)
      .replace(to_replace='\s+$', value='', regex=True)
      .replace(to_replace='^\s+', value='', regex=True)

    .assign(state = lambda df_: df_[('Unnamed: 0_level_0', 'State')].ffill())
    .set_index('state')

    .replace(to_replace='^.*[^\d.].*$', value='', regex=True)  # any cell with non-digits, non-., should be turned into ''
    .drop(columns=[('Unnamed: 0_level_0', 'State'),
                   ('Single Filer', 'Rates.1'),
                   ('Married Filing Jointly', 'Rates.1')])
    .replace(to_replace='\D+', value='', regex=True)
    .replace(to_replace='', value='0')
    .astype(float)
    .ffill()
    .drop_duplicates()
    
)

df

Single Filer            Married Filing Jointly             \
              Rates   Brackets                  Rates   Brackets   
state                                                              
Ala.         0.0200        0.0                 0.0200        0.0   
Ala.         0.0400      500.0                 0.0400     1000.0   
Ala.         0.0500     3000.0                 0.0500     6000.0   
Alaska       0.0000     3000.0                 0.0000     6000.0   
Ariz.        0.0250        0.0                 0.0250        0.0   
...             ...        ...                    ...        ...   
D.C.         0.0650    40000.0                 0.0650    40000.0   
D.C.         0.0850    60000.0                 0.0850    60000.0   
D.C.         0.0925   250000.0                 0.0925   250000.0   
D.C.         0.0975   500000.0                 0.0975   500000.0   
D.C.         0.1075  1000000.0                 0.1075  1000000.0   

       Standard Deduction          Personal Exemption                    
                   Single   Couple             Single  Couple Dependent  
state                                                                    
Ala.               3000.0   8500.0             1500.0  3000.0    1000.0  
Ala.               3000.0   8500.0             1500.0  3000.0    1000.0  
Ala.               3000.0   8500.0             1500.0  3000.0    1000.0  
Alaska                0.0      0.0                0.0     0.0       0.0  
Ariz.             15000.0  30000.0                0.0     0.0       0.0  
...                   ...      ...                ...     ...       ...  
D.C.              15000.0  30000.0                0.0     0.0       0.0  
D.C.              15000.0  30000.0                0.0     0.0       0.0  
D.C.              15000.0  30000.0                0.0     0.0       0.0  
D.C.              15000.0  30000.0                0.0     0.0       0.0  
D.C.              15000.0  30000.0                0.0     0.0       0.0  

[159 rows x 9 columns]

In [47]:
# Which state has the highest rate for single filers? Which states have no income tax at all?

(
    df
    [('Single Filer', 'Rates')]
    .agg(['idxmax', 'max'])
)

idxmax    Calif.
max        0.133
Name: (Single Filer, Rates), dtype: object

In [48]:
(
    df
    [('Single Filer', 'Rates')]
    .loc[lambda s_: s_ == 0]
)

state
Alaska    0.0
Fla.      0.0
Nev.      0.0
S.C.      0.0
S.D.      0.0
Wash.     0.0
Wyo.      0.0
Name: (Single Filer, Rates), dtype: float64